In [2]:
import astropy.table as tb
from astropy.time import Time


In [134]:
dssi_sessions = tb.Table.read("../2024-07-08 Process DSSI olist logs/DSSI sessions.csv")

tic_sessions = dssi_sessions[(sessions["TIC ID"].mask == False) & (sessions["MidTime JD"] > 0)]
tic_sessions

Target Name,TIC ID,Speckle Session,StartTime JD,MidTime JD,EndTime JD,MidTime UTC,Num Sequences
str13,str13,int32,float64,float64,float64,str19,int32
H300038,TIC 328181241,356,2459849.8666666667,2459849.8680555555,2459849.8694444443,2022-09-27 08:50:00,3
H300043,TIC 168789840,364,2459849.929861111,2459849.93125,2459849.9326388887,2022-09-27 10:21:00,3
H300045,TIC 454140642,366,2459849.9381944444,2459849.939236111,2459849.9402777776,2022-09-27 10:32:30,3
H300046,TIC 139650665,373,2459849.961111111,2459849.963194445,2459849.965277778,2022-09-27 11:07:00,3
H300047,TIC 178953404,374,2459849.967361111,2459849.9697916666,2459849.972222222,2022-09-27 11:16:30,5
H300051,TIC 200094011,383,2459850.019444444,2459850.020833333,2459850.0222222223,2022-09-27 12:30:00,3
H300036,TIC 389836747,387,2459850.834027778,2459850.834027778,2459850.834027778,2022-09-28 08:01:00,1
H300032,TIC 283940788,390,2459850.8486111113,2459850.8506944445,2459850.8527777777,2022-09-28 08:25:00,5
H300033,TIC 284814380,392,2459850.8583333334,2459850.8604166666,2459850.8625,2022-09-28 08:39:00,5


In [133]:
ephemerides = tb.vstack([
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2022 Binary Parameters.csv"),
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2023 Binary Parameters.csv"),
])
ephemerides

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 9493888,A,2.098992,1816.2345,1817.283786,146,117,2.9,2.7
TIC 25818450,A,10.132402,1769.9109,1776.391584,12,9,--,--
TIC 27543409,A,2.122862,1493.1001,1494.153889,50,15,--,--
TIC 31928452,A,2.8823,1337.9129,1339.358085,30,25,2.1,1.6
TIC 52856877,A,5.186818,1791.059,1793.652928,220,90,5.5,5.5
TIC 63459761,A,4.244072,1715.1118,1717.168477,15,10,7.0,9.2
TIC 75740921,A,0.93308,1519.6949,1520.16256,87,59,2.7,2.6
TIC 78568780,A,2.88838,1468.5374,1469.962816,57,20,3.9,3.7
TIC 79140936,A,3.54389,1468.3795,1469.78607,23,7,3.7,3.4


In [135]:
eclipses = []
for tic_session in tic_sessions:
    id = tic_session["TIC ID"]
    t = tic_session["MidTime JD"]
    for member in ["A", "B", "C"]:
        ephem = ephemerides[(ephemerides["Local ID"] == id) & (ephemerides["Member"] == member)]
        if len(ephem) == 0:
            continue
        ephem = ephem[0]
        period = ephem["Period"]
        for comp_letter, component in {"a": "Primary", "b": "Secondary"}.items():
            if ephem[f"T0 {component}"] > 0 and ephem[f"Duration {component}"] > 0:
                t0 = float(ephem[f"T0 {component}"]) + 2457000
                half_duration = ephem[f"Duration {component}"] / 24 / 2
                cycles = round((t - t0) / period)
                eclipse_beg = t0 + (period * cycles) - half_duration
                eclipse_end = eclipse_beg + 2 * half_duration
                confirmed = eclipse_beg < t < eclipse_end
                if confirmed:
                    eclipses.append((id, tic_session["Speckle Session"], member, comp_letter, eclipse_beg, eclipse_end, tic_session["StartTime JD"], t, tic_session["EndTime JD"], t0, period, half_duration, (t - t0) / period))
print(len(eclipses), len(tic_sessions))
for eclipse in eclipses:
    print(eclipse)
eclipses

34 161
('TIC 168789840', 364, 'C', 'a', 2459849.829601667, 2459849.9712683335, 2459849.929861111, 2459849.93125, 2459849.9326388887, 2459151.446, 8.217111, 0.07083333333333333, 85.00375010145423)
('TIC 178953404', 374, 'B', 'b', 2459849.898607, 2459850.061107, 2459849.967361111, 2459849.9697916666, 2459849.972222222, 2458477.727164, 28.005157, 0.08125, 48.99964059000981)
('TIC 283940788', 390, 'A', 'b', 2459850.7291343333, 2459850.999967667, 2459850.8486111113, 2459850.8506944445, 2459850.8527777777, 2458766.180072, 0.876867, 0.13541666666666666, 1236.9841976545038)
('TIC 284814380', 392, 'A', 'b', 2459850.850912, 2459851.1134120002, 2459850.8583333334, 2459850.8604166666, 2459850.8625, 2458798.531018, 4.079268, 0.13125, 257.97015510295194)
('TIC 286470992', 417, 'B', 'a', 2459850.5997, 2459851.0122, 2459850.970833333, 2459850.971875, 2459850.972916667, 2458818.6702, 4.128543, 0.20625000000000002, 250.04018972304817)
('TIC 251757935', 418, 'B', 'a', 2459850.912211333, 2459851.108044666

[('TIC 168789840',
  364,
  'C',
  'a',
  2459849.829601667,
  2459849.9712683335,
  2459849.929861111,
  2459849.93125,
  2459849.9326388887,
  2459151.446,
  8.217111,
  0.07083333333333333,
  85.00375010145423),
 ('TIC 178953404',
  374,
  'B',
  'b',
  2459849.898607,
  2459850.061107,
  2459849.967361111,
  2459849.9697916666,
  2459849.972222222,
  2458477.727164,
  28.005157,
  0.08125,
  48.99964059000981),
 ('TIC 283940788',
  390,
  'A',
  'b',
  2459850.7291343333,
  2459850.999967667,
  2459850.8486111113,
  2459850.8506944445,
  2459850.8527777777,
  2458766.180072,
  0.876867,
  0.13541666666666666,
  1236.9841976545038),
 ('TIC 284814380',
  392,
  'A',
  'b',
  2459850.850912,
  2459851.1134120002,
  2459850.8583333334,
  2459850.8604166666,
  2459850.8625,
  2458798.531018,
  4.079268,
  0.13125,
  257.97015510295194),
 ('TIC 286470992',
  417,
  'B',
  'a',
  2459850.5997,
  2459851.0122,
  2459850.970833333,
  2459850.971875,
  2459850.972916667,
  2458818.6702,
  4.